# Creating a training database: import and load Pamguard CSV files


(CLICKLEARN DSTI Project)

The objective of this notebook is to complete Ketos *Creating Database (Extended)* notebook:
https://docs.meridian.cs.dal.ca/ketos/tutorials/create_database/index.html

Sections *1. Imports* and *2. Building annotation files* of this notebook are to be implemented at the beginning of *Chapter 2. Loading the annotations* of the original notebook. Chapter 2 describes methods to import CSV files which already match to the Ketos requirements. No major implementations were provided in Chapter 3 and following ones.

This notebook provides methods to load a CSV file exported by **Pamguard software**, and convert it as Ketos annotation table, dealing with datetime and timedelta operations. Pamguard is an open source software which objective is to provide free and easy to handle tools for cetacean passive acoustic monitoring (PAM):
https://www.pamguard.org/

Two types of CSV file exported by Pamguard can be used with these methods:

* **Events annotation CSV files**: quoting start and end of time ranges (several seconds long) in which clicks streams have been identified.
* **Clicks annotation CSV files**: quoting start time of identified clicks. Click duration is about few milliseconds and has to be set in parameters (see below).

The converted annotation tables will then be ready to be directly used by Ketos library (see Chapter 3 and followings).


## Contents:

[1. Imports](#section1)  
[2. Building annotation files](#section2)  



<a id=section1></a>

<a id="section1"></a>
## 1. Imports

### Importing the packages
(CLICKLEARN DSTI)

We will use several modules within ketos and also the pandas package



In [2]:
import os
import numpy as np
import pandas as pd
from ketos.data_handling import selection_table as sl
import ketos.data_handling.database_interface as dbi
from ketos.data_handling.parsing import load_audio_representation
from ketos.audio.spectrogram import MagSpectrogram
from ketos.data_handling.parsing import load_audio_representation

from datetime import datetime
from typing import Union
import math
import random

# This last package is made by ClickLearn DSTI team to extract informations from a Pamguard CSV export file 
# and buid a Ketos annotation table (Pandas dataframe)
import ketos_annotation_table as kat

C:\Users\groul\anaconda3\envs\ketos_env\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


### Import functions

(CLICKLEARN DSTI Project)

<span style = "color : red;">
TODO:
    
* Move following methods in Ketos_annotation_table.py when finished, then delete this chapter
</span>


In [22]:
def pamguard_name_to_datetime(filename:str):
    """Extract a datetime value from a PAMGUARD sound file name.

        Args:
            filename : str
                PAMGUARD .wav or audio_file name
        Returns:
            datetime
                Formatted datetime value (like 2019-02-24 13:15:00)

        Examples
            >>> pamguard_name_to_datetime('Click_Detector_Sperm_whale_click_detector_Clicks_20180601_010000.pgdf')
            datetime.datetime(2018, 6, 1, 1, 0)
            >>> pamguard_name_to_datetime('192_20180705_123056_853.wav')
            datetime.datetime(2018, 7, 5, 12, 30, 56)
    """
    # split filename from its suffix (.wav or .pgdf)
    filename, suffix = os.path.splitext(filename)
    # remove milliseconds from filename (occurs on Pamguard .wav filenames)
    if ".wav" in suffix:
        filename = filename[:-4]
    # returns the date part of the remaining filename as a datetime
    return datetime.strptime(filename[-15:], '%Y%m%d_%H%M%S')

In [23]:
def strip_dataframe(df):
    """Removes any white space before and after value of every cells of a dataframe.

        Args:
            df: pd.DataFrame
        Returns:
            df: pd.DataFrame
    """
    for column in df:
        if isinstance(df[column][0], str):
            df[column] = df[column].str.strip()
    return df

In [217]:
def update_annotation_df_labels(annotation_df, labels_to_modify:set, valid_labels: set):
    """Replaces a set of labels with valid labels, in a columns named 'label' of a Pandas Annotation dataframe
    
        Args:
            annotation_df: pd.DataFrame
                Annotation table imported from Pamguard
            labels_to_modify: set
                set of strings
            valid_labels: set
                set of strings characters corresponding to every valid labels for any annotation

        Returns:
            annotation_df: pd.DataFrame

    """
    # check validity of new labels
    unmatched_labels = set(labels_to_modify.values()) - set(valid_labels)
    if len(unmatched_labels) != 0:
        print('Following labels doesn\'t match label list: ', unmatched_labels)
    else:
        # replace old labels by new ones
        for old_label, new_label in labels_to_modify.items():
            annotation_df.loc[annotation_df.label == old_label,'label'] = new_label
    return annotation_df

In [25]:
def subsample_dataframe(df_to_subsample, train_proportion:float):
    """Subsample a Pandas DataFrame into 2 others, based on its indexes.
        
        Args:
            df_to_subsample: pd.DataFrame
            train_proportion: float
                proportion (]0;1[) of indexes to subsample in the dataset, in order to build the train dataset
    
        Returns: tuple with 2 dataframes:
            [0] train subsample 
            [1] test subsamples
    """
    length = len(df_to_subsample)
    nb_train_rows = math.ceil(train_proportion*length)
    #create a sequence corresponding to df indexes to subsample 
    sequence = set(range(length))
    train_indexes = random.sample(sequence, nb_train_rows)
    test_indexes = list(sequence - set(train_indexes))
    return df_to_subsample.iloc[train_indexes], df_to_subsample.iloc[test_indexes]

In [40]:
def pamguard_annotations_csv_to_df(annotation_csv_path:str, annotation_type:str = 'events', events_subsampling_option:int = 2, click_duration:float = 2):
    """Convert a Pamguard CSV annotation file into a Ketos annotation table (non standardized). 
        This method contains several identical "if" statements, that were not gathered for more readability.

        Args:
            annotation_csv_path: str
                complete path of the CSV file. CSV separator must be ','
            annotation_type: str
                2 values are possible: 'events' (default value, for Events annotation CSV files) or 'clicks' (for Clicks annotation CSV files)
            events_subsampling_option: int
                Subsampling is optional but highly recommended to avoid unsure detections. 
                Several subsamplings of annotations_csv_df are possible, 
                from the less to the most selective option:
                - 0: no subsampling
                - 1: subsample excluding all commented detections in 'comment' column 
                    (comments always refer to unsure detections), 
                    then including 'definite' ('DLD') and 'probable' ('DLP') dolphin click detections as well
                - 2 (default value): subsample excluding all commented detections in 'comment' column 
                    (comments always refer to unsure detections), 
                    then including only 'definite' dolphin click detections ('DLD')
                N.B.: Clicks annotation CSV files are only based on (uncommented) definite dolphin click 
                detections ('DLD'), so it is unnecessary to subsample them regarding to any comment or label. 
            click_duration: float
               click duration in milliseconds
               
        Returns:
            annotations_df: pd.DataFrame
                Ketos annotation table (non standardized)
    """
    # import CSV in a dataframe
    annotations_df = pd.read_csv(annotation_csv_path, sep=",")

    # Remove all whitespaces from string typed series of the dataframe 
    annotations_df = strip_dataframe(annotations_df)
    
    # SUBSAMPLING (SPECIFIC TO EVENT ANNOTATION DF, with option 2 > 1 > 0, check docstring):
    if annotation_type == 'events':
        if events_subsampling_option == 2:
            annotations_df = annotations_df.loc[((annotations_df.comment.isna()) 
                                                 & (annotations_df.eventType == 'DLD')),]
        if events_subsampling_option == 1:
            annotations_df = annotations_df.loc[(annotations_df.comment.isna()),]    
    
    # DATETIME AND LABELS REWORKING:
    if annotation_type == 'events':
        
        # updating pd.Series type from string to datetime
        annotations_df.EventStart = pd.to_datetime(annotations_df.EventStart, format = '%Y-%m-%d %H:%M:%S.%f')
        
        # To complete Eventstart series, EventStartMilliseconds must be added to Clickstart values as a pd.Timedelta
        EventStartMilliseconds_events = pd.Series([pd.Timedelta(val, unit = 'milliseconds') for val in annotations_df.EventStartMilliseconds])
        # reset index allows to do arithmetic operators between annotations_df and EventStartMilliseconds_events
        annotations_df = annotations_df.reset_index()
        annotations_df.EventEnd = pd.to_datetime(annotations_df.EventEnd, format = '%Y-%m-%d %H:%M:%S.%f')

        # updating all datetime values (beginning & end of events) into seconds from the beginning of the recording
        # for this, convert first file name into pandas Timedelta format
        annotations_df.EventStart = (annotations_df.EventStart 
                                     + EventStartMilliseconds_events
                                     - annotations_df.WAVFile.apply(pamguard_name_to_datetime)).apply(pd.Timedelta.total_seconds)
        annotations_df.EventEnd = (annotations_df.EventEnd 
                                   - annotations_df.WAVFile.apply(pamguard_name_to_datetime)).apply(pd.Timedelta.total_seconds)
        
    if annotation_type == 'clicks':
        
        # updating pd.Series type from string to datetime
        annotations_df.Clickstart = pd.to_datetime(annotations_df.Clickstart, format = '%Y-%m-%d %H:%M:%S.%f')
        
        # To complete Clickstart series, UTCMilliseconds must be added to Clickstart values as a pd.Timedelta
        UTCMilliseconds_clicks = pd.Series([pd.Timedelta(val, unit = 'milliseconds') for val in annotations_df.UTCMilliseconds])
        
        # Converts click_duration into Timedelta format
        click_duration = pd.Timedelta(click_duration, unit = 'milliseconds')
        
        # Compute Clickend series thanks to click duration then add it in the dataframe
        annotations_df['Clickend'] = pd.Series(annotations_df.Clickstart + click_duration)
        
        # add a label for each click detection
        annotations_df['label'] = label

        # updating all datetime values (beginning & end of events) into seconds from the beginning of the recording
        # for this, convert first file name into pandas Timedelta format
        annotations_df.Clickstart = (annotations_df.Clickstart 
                                     + UTCMilliseconds_clicks
                                     - annotations_df.BinaryFile.apply(pamguard_name_to_datetime)).apply(pd.Timedelta.total_seconds)
        annotations_df.Clickend = (annotations_df.Clickend 
                                           + UTCMilliseconds_clicks 
                                           - annotations_df.BinaryFile.apply(pamguard_name_to_datetime)).apply(pd.Timedelta.total_seconds)

    # COLUMNS REWORKING:
    if annotation_type == 'events':
        # select only columns required in Ketos annotation table
        annotations_df = annotations_df.loc[:,('Id', 'EventStart', 'EventEnd', 'eventType', 'WAVFile', 'comment')]
        # update column names to fit Ketos annotation table format
        annotations_df = annotations_df.rename(
            columns = {'EventStart':'start', 'EventEnd':'end', 'eventType':'label', 'WAVFile':'filename'})
    
    if annotation_type == 'clicks':
        # select only columns required in Ketos annotation table
        annotations_df = annotations_df.loc[:,('Id', 'Clickstart', 'Clickend', 'label', 'BinaryFile')]
        # update column names to fit Ketos annotation table format
        annotations_df = annotations_df.rename(
            columns = {'Clickstart':'start', 'Clickend':'end', 'BinaryFile':'filename'})

    # Update pamguard default labels with labels used by Ketos
    annotations_df = update_annotation_df_labels(annotations_df, labels_to_modify, valid_labels)
    
    # Check start and end values of events annotation df
    if annotation_type == 'events':
        check_pamguard_annotation_df(annotations_df)
        

    return annotations_df

In [41]:
def check_pamguard_annotation_df(df):
    """Check annotations from a CSV file extracted from Pamguard software. Uses check_annotation in a loop.
    
        Args:
            df: Pandas DataFrame
                Pamguard annotation DataFrame converted by the pamguard_annotations_csv_to_df() method.
        
        Returns: 
            errors: list
                List of errors containing each incorrect annotation
    """
    errors = []
    for index, row in df.iterrows():
        check_annotation_time(row.start, row.end, row.Id)

In [28]:
def check_annotation_time(start:str, end:str, index = 0):
    """Check time validity of 1 annotation (or 1 row of an annotation table): end value must be subsequent to start value.
        Print error messages.
        
        Args:
            start: str
                Start time for the annotation, in seconds from the beginning of the file
            end: str
                End time for the annotation, in seconds from the beginning of the file
            index: int
                Row index (DataFrames)
        
        Returns: 
            str
               String, containing index and details about the incorrect annotation
    
    """
    if start > end: 
        return f'Id {index}: end value ({end}) is prior to start value ({start})'

##### Hand input methods

In [29]:
def check_hand_input(start: Union[int, float, str], end: Union[int, float, str], label: str, valid_labels: set):
    """Check validity of an annotation added by hand. This checking is used before appending it in the annotation 
        dataframe with append_hand_input_to_annotations() method.     
        Each annotation must pass following tests before being considered as correct :
        - start and end types must be integer, float or string
        - label must be included in kat.labels list
        
        Args:
            start: int, float, str
                Start time for the annotation, in seconds from the beginning of the file
            end: int, float, str
                End time for the annotation, in seconds from the beginning of the file
            label: str
                Label of the annotation
            valid_labels: set
                Set of every valid labels (strings characters) for any annotation 
        
        Returns: 
            errors: list
                List of errors (string characters) describing each incorrect annotation.
    """
    #initialize error list to be returned
    errors = []

    #check validity of start & end values 
    try:
        float(start)
    except:
        errors += ['start value is not int, float or string']
    try:
        float(end)
    except:
        errors += ['end value is not int, float or string']
    if start > end: 
        errors += ['end value is prior to start value']
    
    #test if label is included in kat.labels list
    if label not in valid_labels:
        errors += ['label is not included in valid labels list']
        
    return errors

In [206]:
def check_overlap_errors(annotation_to_add, annotation_df):
    """Check if a hand mande annotation start or end value overlaps one or several annotations of a Pandas DataFrame.
        
        Args:
            annotation_to_add: Pandas Series
                Annotation to add in the annotation table
            annotation_df: Pandas DataFrame
                Ketos annotation table
        
        Returns: 
            errors: list
                List of errors (string characters) describing each incorrect annotation.
    """
    errors = []
    for index, row in annotation_df.iterrows():
        if (row.start <= annotation_to_add.iloc[0]['start'] <= row.end):
            errors += [f'start value overlap at index {index}']
        if (row.start <= annotation_to_add.iloc[0]['end'] <= row.end):
            errors += [f'end value overlap at index {index}']
    return errors

In [218]:
def append_hand_input_to_annotations(annotation_dataframe, filename:str, start:float, end:float, label:str, valid_labels:set):
    """Appends a custom annotation to a Ketos annotation table. If any error occurs, print the error description. 
        
        Args:
            annotation_dataframe: Pandas DataFrame
                Ketos annotation table
            filename: str
                Name of the audio file
            start:float
                Start time for the annotation, in seconds from the beginning of the file
            end: float
                End time for the annotation, in seconds from the beginning of the file
            label: str
                Label of the annotation
            valid_labels: set
                Set of every valid labels (strings characters) for any annotation 
        
        Returns: 
            annotation_dataframe: Pandas DataFrame
                Updated Ketos annotation table with the new annotation 
    """
    errors = check_hand_input(start, end, label, valid_labels)
    
    if not errors:
        Id = annotation_dataframe.iloc[-1]['Id'] + 1
        annotation_temp = pd.DataFrame([[Id , round(float(start),4), round(float(end),4), label, filename]], 
                                       columns = ['Id', 'start', 'end', 'label', 'filename']) 
        errors = check_overlap_errors(annotation_temp, annotation_dataframe)
        if not errors:
            #append temp to annotation DF 
            annotation_dataframe = annotation_dataframe.append(annotation_temp, ignore_index = True)
    if errors:
        print('Following errors occured:')
        for er in errors:
            print(er)
    return annotation_dataframe

<a id="section2"></a>
## 2. Building annotation files


This section describes methods to create or modify an annotation table. New methods are added to the original documentation to import Pamguard CSV files and to add annotations by hand.

### Setting parameters to build the Annotation table 
(CLICKLEARN DSTI Project)


This part contains all required settings to build annotation dataframes:
* Audio files path (where all audio files are stored)
* Annotation file path, where: 
    + CSV fils are stored
    + Annotation tables **will be** stored
* Click duration in milliseconds (for Click annotation CSV files)
* Set of valid labels to be used (all labels from CSV file that are not matching this set will be stored in an error log file)

In [3]:
# PATH SETTINGS
# Annotation CSV file folder path (MAC)
# annotations_folder_path ='/Users/benoitmialet/Ketos/tutorials_create_database_database_creation_tutorial'
# Annotation CSV file folder path (W10)
annotations_folder_path = (r'D:\SYSTEL\Ketos\tutorials_create_database_database_creation_tutorial')
# Annotation CSV file name
csv_events_name = 'UBISEA_acoustic_detections_samples_events_completed.csv' # WAVFile column was manually added in this file 
csv_clicks_name = 'UBISEA_acoustic_detections_samples_clicks.csv'

# PATH BUILDING
csv_folder_path = annotations_folder_path
csv_events_path = os.path.join(csv_folder_path,csv_events_name)
csv_clicks_path = os.path.join(csv_folder_path,csv_clicks_name)

# CLICK ANNOTATION SETTINGS 
# set click duration (milliseconds) for the dataframe
clickDuration = pd.Timedelta(2.5, unit = 'milliseconds')
# set label corresponding to a click detection
label = 'DLD'

# LABELS SETTINGS
# valid labels for annotation dataframes, must be a list:
valid_labels = {'dolphin_click','other'}
# annotation dataframe labels to replace by valid labels (must be included in valid labels)
labels_to_modify = {'DLP':'dolphin_click','DLD':'dolphin_click'}

### 2-Option 1: Importing a Pamguard CSV file
(CLICKLEARN DSTI Project)

CSV files exported by Pamguard software can be converted into a Ketos annotation table.
Export must be done with **default options and variable names**. 2 CSV file types are possible to import, with 'pamguard_annotations_csv_to_df' method : 
* **"events"**: in these files, annotations correspond to groups of clicks of several seconds.
    + Between EventStart and EventEnd datetime values, a various number of clicks (nClicks) are included.
    + use method with parameters annotation_type == 'events' and  events_subsampling_option (report to the method's docstring)
    + In the CSV files, columns **in bold** are required and must have **exactely the same headers** as followig: 
![Pamguard event csv](capture_PAMGUARD_event_csv.png)


* **"clicks"**: in these files, annotations correspond to beginning of clicks ('Clickstart', in datetime format). 'Clickend' values end are automatically computed by setting a click duration ('click_duration') in milliseconds among parameters. 
    Clickend will simply be the result of Clickstart + click_duration datetimes. In the CSV files, columns **in bold** are required and must have **exactely the same headers** as followig:
![Pamguard click csv](capture_PAMGUARD_click_csv.png)


#### Convert CSV annotation files into Ketos annotation tables
pamguard_annotations_csv_to_df() and pamguard_annotations_csv_to_df() methods are built with a set of other methods to import and convert a Pamguard CSV file into a Ketos annotation Table. The returned data frame are not yet standardized and thus can be modified or processed.

In [43]:
events_annot_df = pamguard_annotations_csv_to_df(csv_events_path, 'events', events_subsampling_option = 2)
clicks_annot_df = pamguard_annotations_csv_to_df(csv_clicks_path, 'clicks', click_duration = 2.5)

In [44]:
events_annot_df.head(5)

,Id,start,end,label,filename,comment
0,465,25968.249,26012.0,dolphin_click,192_20180601_010000_458.wav,NaN
1,466,26699.235,26726.0,dolphin_click,192_20180601_010000_458.wav,NaN
2,469,66225.582,66262.0,dolphin_click,192_20180601_010000_458.wav,NaN
3,473,67330.013,67353.0,dolphin_click,192_20180601_010000_458.wav,NaN
4,474,67522.346,67577.0,dolphin_click,192_20180601_010000_458.wav,NaN


In [39]:
events_annot_df.head(5)

,Id,start,end,label,filename,comment
0,465,25968.249,26012.0,dolphin_click,192_20180601_010000_458.wav,NaN
1,466,26699.235,26726.0,dolphin_click,192_20180601_010000_458.wav,NaN
2,469,66225.582,66262.0,dolphin_click,192_20180601_010000_458.wav,NaN
3,473,67330.013,67353.0,dolphin_click,192_20180601_010000_458.wav,NaN
4,474,67522.346,67577.0,dolphin_click,192_20180601_010000_458.wav,NaN


#### Standardize Ketos annotation tables
ketos.data_handling.selection_table.standardize() is the original method to standardize an annotation table for Ketos once it is built. ketos.data_handling.selection_table.is_standardized() checks if this standardization is completed or return error description.

In [119]:
# standardize all annotation tables
std_events_annot_df = sl.standardize(table = events_annot_df, signal_labels = valid_labels, trim_table=False)
std_clicks_annot_df = sl.standardize(table = clicks_annot_df, signal_labels = valid_labels, trim_table=False)


print('check if both events and clicks files are in standardized format:\n', 
     sl.is_standardized(std_events_annot_df), '\n',
     sl.is_standardized(std_clicks_annot_df))

check if both events and clicks files are in standardized format:
 True 
 True


#### Splitting into 2 random subsamples (annotation_train / annotation_test)
(CLICKLEARN DSTI Project)
subsample_dataframe() method is created to split the annotation table into 2 subsamples:
* Train annotation table (Ketos will use it to create the training dataset)
* Test annotation table (Ketos will use it to create the testing dataset)

In [71]:
help(subsample_dataframe)

Help on function subsample_dataframe in module __main__:

subsample_dataframe(df_to_subsample, train_proportion: float)
    Subsample a Pandas DataFrame into 2 others, based on its indexes.
    
    Args:
        df_to_subsample: pd.DataFrame
        train_proportion: float
            proportion (]0;1[) of indexes to subsample in the dataset, in order to build the train dataset
    
    Returns: tuple with 2 dataframes:
        - [0] train subsample 
        - [1] test subsamples



In [38]:
std_annot_train_events = subsample_dataframe(std_events_annot_df, 0.7)[0]
std_annot_test_events = subsample_dataframe(std_events_annot_df, 0.7)[1]

std_annot_train_clicks  = subsample_dataframe(std_clicks_annot_df, 0.7)[0]
std_annot_test_clicks = subsample_dataframe(std_clicks_annot_df, 0.7)[1]

In [39]:
std_annot_train_events.head(5)

Id      start      end  label comment
filename                    annot_id                                        
192_20180603_000000_458.wav 1         527   5258.439   5269.0      2     NaN
192_20180601_010000_458.wav 21        501  73691.091  73703.0      2     NaN
                            9         483  72607.571  72620.0      2     NaN
                            18        496  73415.745  73438.0      2     NaN
                            6         478  71702.295  71713.0      2     NaN

In [40]:
std_annot_train_clicks.head(5)

,,Id,start,end,label
filename,annot_id,,,,
Click_Detector_Sperm_whale_click_detector_Clicks_20180601_190000.pgdf,525,178620,2239.421,2239.4235,2
Click_Detector_Sperm_whale_click_detector_Clicks_20180601_230000.pgdf,91,181713,362.053,362.0555,2
Click_Detector_Sperm_whale_click_detector_Clicks_20180603_000000.pgdf,13,182027,1381.822,1381.8245,2
Click_Detector_Sperm_whale_click_detector_Clicks_20180601_190000.pgdf,503,182408,2234.209,2234.2115,2
Click_Detector_Sperm_whale_click_detector_Clicks_20180601_210000.pgdf,1340,180233,1459.994,1459.9965,2


**The train and test files are now ready to be used in step *5. Creating segments of uniform length* and following ones from the original Notebook.**





### 2-Option 2: adding annotations by hand (not likely to be used)
(CLICKLEARN DSTI Project)

This part allows to add custom annotation by hand to a Ketos annotation table.

If used on a **standardized** Ketos annotation table, numerical labels must be used.

##### Report each annotation here, and then run next cell to append the annotation into the dataframe:

*    **start**: start time for the annotation, in seconds from the beginning of the file
*    **end**: end time for the annotation, in seconds from the beginning of the file
*    **label**: label for the annotation

In [213]:
# type annotation details here

annotations_df = events_annot_df
filename = '192_20180601_010000_458.wav'
start = 527.55  #in seconds
end = 528.32    #in seconds
label = 'other' #annotation label

In [214]:
# append the annotation to an annotation dataframe

annotations_df = append_hand_input_to_annotations(annotations_df, filename, start, end, label, valid_labels)
annotations_df.tail(5)

,Id,start,end,label,filename,comment
31,521,10157.910,10180.00,dolphin_click,192_20180602_000000_458.wav,NaN
32,522,1378.201,1390.00,dolphin_click,192_20180603_000000_458.wav,NaN
33,527,5258.439,5269.00,dolphin_click,192_20180603_000000_458.wav,NaN
34,528,5660.552,5672.00,dolphin_click,192_20180603_000000_458.wav,NaN
35,529,527.550,528.32,other,192_20180601_010000_458.wav,NaN


### 2-Option 3: Importing a CSV file already formatted for Ketos
(Text from original Notebook)

Our annotations are saved in two `.csv` files (with values separated by `;`): `annotations_train.csv` and `annotations_test.csv`, which we will use to create the training and test datasets respectively. These files can also be found within the `.zip` file at the top of the page. 

In [177]:
annot_train_path = os.path.join(*annotations_folder_path.split('\\'),"annotations_train.csv")
annot_train = pd.read_csv(annot_train_path, sep = ",")
annot_test_path = os.path.join(*annotations_folder_path.split('\\'),"annotations_test.csv")
annot_test = pd.read_csv(annot_test_path, sep = ",")

Let's inspect our annotations

The **annot_train** dataframe contains 1000 rows and the **annot_test** 500.
The columns indicate:

**start:** start time for the annotation, in seconds from the beginning of the file  
**end:** end time for the annotation, in seconds from the beginning of the file   
**label:** label for the annotation (in our case, all annotated signals are 'upcalls', but the origincal DCLDE2013 dataset also had 'gunshots')  
**sound_file:** name of the audio file  
**datetime:** a timestamp for the beginning of the file (UTC)  

---
**Starting from here, following text is taken from the orginial Notebook, with no major implementation from ClickLearn DSTI project team. 
Only some line of codes which are specific to the project needs were added.**

<a id=section3></a>

### 3. Putting the annotations in the Ketos format (to skip with Pamguard CSV importation)


Let's check if our annotations follow the Ketos standard.

If that's the case, the function ```sl.is_standardized``` will return ```True```. 


In [180]:
sl.is_standardized(annot_train)

Setting the *verbose* argument to ```False``` will not show the example above:

In [126]:
sl.is_standardized(annot_test, verbose=False)

Neither of our annotations are in the format ketos expects. But we can use the ```sl.standardize``` function to convert to the specified format.

The *annot_id* column is created automatically by the ```sl.standardize``` function. From the remaining required columns indicated in the example above, we already have *start*, *end* and *label*. Our *sound_file* column needs to be renamed to *filename*, so we will need to provide a dictionary to specify that. 

We have one extra column, *datetime*, that we don't really need to keep, so we'll set ```trim_table=True```, which will discard any columns that are not required by the standardized.

If we wanted to keep the datetime (or any other columns), we would just set ```trim_table=False```. One situation in which you might want to do that is if you need this information to split a dataset into train/test or train/validation/test, because then you can sort all your annotations by time and make sure the training set does not overlap with the validation/test. But in our case, the annotations are already split.

In [113]:
map_to_ketos_annot_std = {'sound_file': 'filename'} 
std_annot_train = sl.standardize(table=annot_train, signal_labels=["upcall"], mapper=map_to_ketos_annot_std, trim_table=True)
std_annot_test = sl.standardize(table=annot_test, signal_labels=["upcall"], mapper=map_to_ketos_annot_std, trim_table=True)


Let's have a look at our standardized tables

Notice that the 'label' column now encodes 'upcall' as ones (1), as the ketos format uses integers to represent labels.

<a id=section4></a>

### 5. Creating segments of uniform length

If you look back at our `` std_annot_train`` and ```std_annot_test``` you'll notice that annotations have a variety of lengths, since they mark the beginning and end of an upcall and these have variable durations. For our purposes, we want each signal in the database to be represented as spectrograms, all of same length. Each spectrogram will be labelled as containing an upcall or not. 

The ```sl.select``` function in ketos can help us to do just that: for each annotated upcall, it will select a portion of the recording surrounding it. It takes a standardized annotatoin table as input and lets you specify the length of the output segments. We'll use 3 seconds, as it is enough to encompass most upcalls.

Our standardized tables only contain annotated upcalls. Later we will also want some examples of segments that only contain background noise, but for now we'll just create the uniform upcall segments, which we'll call 'positives'


(CLICKLEARN DSTI Project add:)
* **length** (seconds): uniform length of a segment
* **step** (seconds): shift between 2 segments (no shift if length == step)
* **min_overlap** (proportion): if a segment crosses the start or the end of annotation, 
    proportion of its length that is allowed to cross (1 means no cross allowed)

In [43]:
#Clicklearn
positives_train_events = sl.select(annotations=std_annot_train_events, length=0.5, step=0.5, min_overlap=1, center=False, discard_long=False, keep_id=False)
positives_test_events = sl.select(annotations=std_annot_test_events, length=0.5, step=0.5, min_overlap=1, center=False, discard_long=False, keep_id=False)

positives_train_clicks = sl.select(annotations=std_annot_train_clicks, length=0.0025, step=0.0025, min_overlap=1, center=False, discard_long=False, keep_id=False)
positives_test_clicks = sl.select(annotations=std_annot_test_clicks, length=0.0025, step=0.0025, min_overlap=1, center=False, discard_long=False, keep_id=False)

In [100]:
#Ketos documentation
positives_train = sl.select(annotations=std_annot_train, length=3.0)
positives_test = sl.select(annotations=std_annot_test, length=3.0, step=0.0, center=False)


Have a look at the results and notice how each entry is now 3.0 seconds long.

<a id=section5></a>

### 5. Augmenting the data

Data augmentation is a set of tecnhiques used in machine learning to increase the data available to train models. There are many different techniques that can be used. The ```sl.select``` function we just used offers a simple way to augment the data while you are creating the uniform selections. It creates segments that are longer than the annotated signals and then shifts the start and end of those segments, resulting in multiple segments with the same annotated signal (our upcalls) positioned at different times. This is a very safe technique, as it is not altering the original signal, but it can already help to increase the amount of data available. It also helps to present a larger variety of contexts in which the upcall can appear.  

We'll augment the training portion of our annotations by using two additional arguments. The ``step`` specifies how much the signal will be shifted (in seconds). Smaller values will produce more augmented selections, but they will be more similar to the previous selection. The ```min_overlap``` argument specifies the fraction of the augmented signal that needs to overlap the original annotation in order for it to be included in the augmented selections table. A value of 1.0 means 100%, this is, the new annotation will only be included if the entire upcall falls within the stablished interval. Lower values will result in segments that only contain part of the original upcall. We'll set this value to 0.5, meaning that some of our augmented segments might have as little as half of the original call.

In [ ]:
positives_train = sl.select(annotations=std_annot_train, length=3.0, step=0.5, min_overlap=0.5, center=False)


In [ ]:
positives_train

Notice that now our ``positives_train`` tables has almost 3x more rows than before.

<a id=section6></a> 

### 6. Including background noise

Now that we have the positive instances that we need to create our database, we need to include some examples of the negative class, or instances without upcalls.

The ```sl.create_rndm_backgr_selections``` is ideal for our situation. It takes a standardized ketos table describing all sections of the recordings that contain annotations and takes samples from the non-annotaded portions of the files, assuming everything that is not annotated can be used as a 'background' category.

**Note**:
You might find yourself in a different scenario. For example, your annotations might already include a 'background' class or you might have annoted different classes of sounds and you only want to use a few of them. In any case, ketos provides a variety of other functions that are helpful in different scenarios. Have a look at the documentation for more details. Specially the ```selection_table``` module.



In [117]:
file_durations_train = sl.file_duration_table('data/train')
annotations_folder_path

'/Users/benoitmialet/Ketos/tutorials_create_database_database_creation_tutorial'

In [ ]:
##
sl.create_rndm_backgr_selections(std_annot_test, len(std_annot_test), 12, annotations=None, no_overlap=False, trim_table=False)

The ```sl.create_rndm_backgr_selections``` also needs the duration of each file, which we can generate using the ```sl.file_duration``` function.

In [ ]:
file_durations_train = sl.file_duration_table('data/train')
file_durations_test = sl.file_duration_table('data/test') 

In [ ]:
file_durations_train

Now that we have the file durations, we can generate our table of negative segments. We'll specify the same length (3.0 seconds). The ```num``` argument specifies the number of background segments we would like to generate. Let's make this number equal to the number of positive examples in each dataset (```len(positive_train)``` and ``` len(positive_test)```)

In [ ]:
negatives_train=sl.create_rndm_backgr_selections(annotations=std_annot_train, files=file_durations_train, length=3.0, num=len(positives_train), trim_table=True)
negatives_train

In [ ]:
negatives_test=sl.create_rndm_backgr_selections(annotations=std_annot_train, files=file_durations_test, length=3.0, num=len(positives_test), trim_table=True)
negatives_test

There we have it! Now we'll just put the ```positives_train``` and ```negatives_train``` together and do the same to the test tables.

In [ ]:
selections_train = positives_train.append(negatives_train, sort=False)
selections_test = positives_test.append(negatives_test, sort=False)

In [ ]:
selections_train

In [ ]:
selections_test

At this point, we have defined *which* audio segments we want in our database: a little over 5500 in the training dataset, 50% with upcalls and 50% without, and 1000 for the test set, maintaining the same ratio.

Now we need to decide *how* these segments will be represented.

<a id=section7></a>

###  7. Choosing the spectrogram settings

As mentioned earlier, we'll represent the segments as spectrograms.
In the .zip file where you found the data, there's also a spectrogram configuration file (```spec_config.json```) which contains the settings we want to use.

This configuration file is simply a text file in the ```.json``` format, so you could make a copy of it, change a few parameters and save several settings to use later or to share the with someone else.


In [ ]:
spec_cfg = load_audio_representation('spec_config.json', name="spectrogram")

In [ ]:
spec_cfg

The result is a python dictionary. We could change some value, like the step size:

In [ ]:
#spec_cfg['step'] = 0.064

But we will stick to the original here.

<a id=section8></a>

### 8. Creating the database

Now we have to compute the spectrograms following the settings above for each selection in our selection tables and then save them in a database.

All of this can be done with the ```dbi.create_database``` function in Ketos.

We will start with the training dataset. We need to indicate the name for the database we want to create, where the audio files are, a name for the dataset, the selections table and, finally the audio representation. As specified in our ``spec_cfg``, this is a Magnitude spectrogram, but ketos can also create databases with Power, Mel and CQT spectrograms, as well as time-domain data (waveforms).


In [ ]:
dbi.create_database(output_file='database.h5', data_dir='data/train',
                               dataset_name='train',selections=selections_train,
                               audio_repres=spec_cfg)
                              

And we do the same thing for the test set. Note that, by specifying the same database name, we are telling ketos that we want to add the test set to the existing database.

In [ ]:
dbi.create_database(output_file='database.h5', data_dir='data/test',
                               dataset_name='test',selections=selections_test,
                               audio_repres=spec_cfg)
                              

Now we have our database with spectrograms representing audio segments with and without the North Atlantic Right Whale upcall. The data is divided into 'train' and 'test'. 



In [ ]:
db = dbi.open_file("database.h5", 'r')

In [ ]:
db

Here we can see the data divided into 'train' and 'test' These are called 'groups' in HDF5 terms. Within each of them there's a dataset called 'data', which contains the spectrograms and respective labels.

In [ ]:
db.close() #close the database connection

You will likely not need to directly interact with the database. In a following tutorial, we will use Ketos to build a deep neural network and train it to recognize upcalls. Ketos handles the database interactions, so we won't really have to go into the details of it, but if you would like to learn more about how to get data from this database, take a look at the [database_interface](https://docs.meridian.cs.dal.ca/ketos/modules/data_handling/database_interface.html) module in ketos and the [pyTables](https://www.pytables.org/index.html) documentation.